
# Piping Chains and the RunnablePassthrough Class

In [1]:
%load_ext dotenv
%dotenv

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [5]:
RunnablePassthrough().invoke([1,2,3])

[1, 2, 3]

In [8]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} need?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [9]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} need?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [10]:
chat = ChatOpenAI(model_name = "gpt-4",
                  model_kwargs = {"seed": 365},
                  temperature = 0,
                  max_tokens = 100)

C:\Users\Asus\.conda\envs\Langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [11]:
string_parser = StrOutputParser()

In [17]:
chain_tools = chat_template_tools | chat | string_parser | {'tools': RunnablePassthrough()}
chain_strategy = chat_template_strategy | chat | string_parser

In [18]:
print(chain_tools.invoke({"job title": "data scientist"}))

{'tools': '1. Python\n2. R Programming\n3. SQL\n4. Tableau\n5. Hadoop'}


In [16]:
print(chain_strategy.invoke({"tools": '''
1. Python
2. R Programming
3. SQL 
4. Tableau
5. Hadoop
'''}))

1. Python: Start with the basics of Python, such as variables, data types, operators, and control flow. Once you have a good understanding of these, move on to more complex topics like functions, classes, and error handling. Use online resources like Codecademy, Coursera, or Python's official documentation to learn. Practice coding regularly, work on small projects, and gradually move on to more complex ones. 

2. R Programming: Start with the basics of R, such


In [19]:
chain_combined = chain_tools | chain_strategy

In [20]:
print(chain_combined.invoke({"job title": "data scientist"}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like HackerRank or LeetCode. Once you're comfortable with the basics, move on to more advanced topics like object-oriented programming, data structures, and algorithms. Then, explore libraries like NumPy, Pandas, and Matplotlib for data analysis and visualization. 




In [21]:
chain_long = (chat_template_tools | chat | string_parser | {"tools": RunnablePassthrough()} | 
              chat_template_strategy | chat | string_parser)

# Graphing Runnables

In [23]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       